In [56]:
from __future__ import division
import pandas as pd
import numpy as np
from multiprocessing import Pool
import os.path as path
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from skimage import io, transform

import torch

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import pdb

% matplotlib inline

In [5]:
# https://www.kaggle.com/c/landmark-recognition-challenge/data

In [6]:
pwd = "/home/edwin/Datasets/competitions/landmark-recognition-challenge"

In [7]:
data_dir = f"{pwd}/data"

In [8]:
train_csv = f"{pwd}/train.csv"

In [9]:
train_csv

'/home/edwin/Datasets/competitions/landmark-recognition-challenge/train.csv'

In [16]:
df = pd.read_csv(train_csv)

In [ ]:
for i, row in df.iterrows():
    print(row)
    if i == 0:
        break

In [ ]:
splits = np.array_split(df, 512)

In [ ]:
def download_image(dataframe):
    for index, row in dataframe.iterrows():
        image_id = index
        filepath = f"{data_dir}/{row['id']}.jpg"
        if path.exists(filepath):
            print("File already exists")
        else:
            ! wget -O "{filepath}" "{row['url']}"

In [ ]:
p = Pool(512)

In [ ]:
p.map(download_image, splits)

In [17]:
class GoogleLandmarkDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image_id = train_ds.df.iloc[idx].id
        file = f"{data_dir}/{image_id}.jpg"
        try:
            img = Image.open(file)
            img = np.asarray(img)
        except Exception as error:
            img = np.zeros([1200, 1600, 3])
        label = self.df.landmark_id.iloc[idx]
        sample = {
            "image": img,
            "label": label
        }
        if self.transform:
            sample = self.transform(sample)
        return sample

In [18]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image = sample["image"]
        label = sample["label"]

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (self.output_size, self.output_size), mode='constant')
        return {
            "image": img,
            "label": label
        }
    
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image = sample["image"]
        label = sample["label"]

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        
        image = torch.from_numpy(image).float()
        label = torch.from_numpy(np.array([label]))
        
        return {
            "image": image,
            "label": label
        }
    

In [19]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 10, 5)
        self.fc1 = nn.Linear(810 , 4000)
        self.fc2 = nn.Linear(4000, 1000)
        self.fc3 = nn.Linear(1000, 14951)
        
    def forward(self, x):
        # original input torch.Size([4, 3, 75, 75])
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x
    
cnn = CNNModel()

In [58]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01, momentum=0.9)

In [21]:
classes = tuple((set(np.sort(df.landmark_id.values))))

In [15]:
len(classes)

NameError: name 'classes' is not defined

In [22]:
train_set, test_set = train_test_split(df, train_size=1000, test_size=200, random_state=42)

In [25]:
test_train_set, train_valid_set = train_test_split(train_set, train_size=100, test_size=20, random_state=42)

In [26]:
len(test_train_set)

100

In [27]:
len(train_set)

1000

In [29]:
composed = transforms.Compose([Rescale(50), ToTensor()])

In [30]:
train_ds = GoogleLandmarkDataset(df=train_set, transform=composed)
test_ds = GoogleLandmarkDataset(df=test_train_set, transform=composed)

In [57]:
trainloader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=4)
testloader = DataLoader(test_ds, batch_size=64, shuffle=True, num_workers=4)

In [32]:
dataiter = iter(trainloader)

Process Process-2:
Process Process-1:
Process Process-4:
Process Process-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/edwin/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/home/edwin/anaconda3/lib/python3.6/multiprocessing

In [33]:
dataiter.next()

{'image': 
 (0 ,0 ,.,.) = 
   0.1340  0.0937  0.1250  ...   0.0926  0.2234  0.3853
   0.1604  0.2737  0.2895  ...   0.3483  0.3314  0.3463
   0.1049  0.3570  0.3842  ...   0.2586  0.3847  0.3857
            ...             ⋱             ...          
   0.0516  0.0586  0.0525  ...   0.0198  0.0902  0.1067
   0.0331  0.0320  0.0314  ...   0.0174  0.0790  0.0729
   0.0264  0.0306  0.0206  ...   0.0284  0.0752  0.0667
 
 (0 ,1 ,.,.) = 
   0.0955  0.0575  0.0552  ...   0.0416  0.1120  0.3542
   0.1116  0.1326  0.2175  ...   0.3088  0.2920  0.3173
   0.0721  0.2984  0.3626  ...   0.2248  0.3451  0.3523
            ...             ⋱             ...          
   0.0359  0.0377  0.0321  ...   0.0198  0.0680  0.0800
   0.0288  0.0281  0.0220  ...   0.0233  0.0595  0.0533
   0.0186  0.0226  0.0128  ...   0.0226  0.0708  0.0471
 
 (0 ,2 ,.,.) = 
   0.0859  0.0406  0.0450  ...   0.0328  0.0903  0.3200
   0.1018  0.0976  0.1671  ...   0.2719  0.2573  0.2955
   0.0684  0.2391  0.3479  ...   0.1800  

In [34]:
sample = dataiter.next()

In [40]:
i = Variable(sample['image'].cuda())

In [36]:
cnn = CNNModel()

In [43]:
outputs = cnn(i)

In [44]:
_, predicted = torch.max(outputs.data, 1)

In [46]:
sample['label']


 11061
  6696
  9770
 13378
[torch.LongTensor of size 4x1]

In [45]:
predicted.cpu()


  4053
 11234
  7737
  7737
[torch.LongTensor of size 4]

In [60]:
iter = 0
for epoch in range(num_epoch):
    print("Epoch:{}".format(epoch))
    for i, data in enumerate(trainloader, 0):
        images = Variable(data["image"].cuda())
        labels = Variable(data["label"].cuda())

        optimizer.zero_grad()

        outputs = cnn(images)
        loss = criterion(outputs, labels.view(-1))
        loss.backward()
        optimizer.step()
        iter += 1
        if iter % 500 == 0:
            correct = 0
            total = 0
            for load_data in testloader:
                test_images = Variable(load_data['image'].cuda())
                labels = load_data['label']

                test_outputs = cnn(test_images)

                _, predicted = torch.max(test_outputs.data, 1)

                total += labels.size(0)
                correct += (predicted.cpu() == labels.view(-1)).sum()
            accuracy = 100 * correct / total
            print('Correct:{}, Total:{}'.format(correct, total))
            print('Iteraton:{}, Loss:{}, Accuracy: {}'.format(iter, loss.data[0], accuracy))
        

Epoch:0
Epoch:1


In [61]:
print('Iteraton:{}, Loss:{}, Accuracy: {}'.format(iter, loss.data[0], accuracy))

Iteraton:32, Loss:8.652276992797852, Accuracy: 6.0


In [49]:
num_epoch=2

In [ ]:
len(train_ds)

In [ ]:
train_ds[0]

In [ ]:
train_ds.df.iloc[0].id

In [ ]:
plt.imshow(train_ds[0][0])

In [ ]:
test_ds[0][0]